In [114]:
import pandas as pd
import numpy as np

In [116]:
PATH_ROSTER = "D:\\Run Analysis\\blob_storage\\roster\\35th_2021.xlsx"
PATH_SAL = "D:\\Run Analysis\\blob_storage\\driver_salary\\processed\\35th_2021.xlsx"
sal_df = pd.read_excel(PATH_SAL)
# Employee_ID	Hours	 Gross 	Super
roster_df = pd.read_excel(PATH_ROSTER)

sal_df.Employee_ID = sal_df.Employee_ID.str.lower()
sal_df['total'] = sal_df.Gross + sal_df.Super
count_pri_dr = roster_df.groupby(['Primary_employeeID'])['Date'].count().rename("pri_dr")

count_sec_dr = roster_df.groupby(['Secondary_employeeID'])['Date'].count().rename("sec_dr")

count_dr_shifts = pd.concat([count_pri_dr, count_sec_dr], axis=1).fillna(0)

count_dr_shifts['Total'] = count_dr_shifts.pri_dr + count_dr_shifts.sec_dr

count_dr_shifts = count_dr_shifts['Total'].reset_index().rename(columns={"index" : "Employee_ID", "Total" : "total_shifts"})   
sal_shifts_df = sal_df.merge(count_dr_shifts, how="left", on="Employee_ID")

sal_shifts_df['per_shift_pay'] = sal_shifts_df.total / sal_shifts_df.total_shifts
sal_per_shift = sal_shifts_df[["Employee_ID", "per_shift_pay"]]

def pri_sal_cost(df):
    emp_id = df.Primary_employeeID
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

def sec_sal_cost(df):
    emp_id = df.Secondary_employeeID
    sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values
    if len(sal) != 0:
       return sal[0]
    else:
        return 0

roster_df['pri_sal'] = roster_df.apply(pri_sal_cost, axis=1)
roster_df['sec_sal'] = roster_df.apply(sec_sal_cost, axis=1)

roster_df['sal_cost'] = roster_df.pri_sal + roster_df.sec_sal

sal_df = roster_df[['Date','sal_cost','Primary_route', 'Satellite_Route_1','Satellite_Route_2']].fillna(0)    

def per_route_cost(df):
    count = 0 
    p_route = df.Primary_route
    route_1 = df.Satellite_Route_1
    route_2 = df.Satellite_Route_2
    if p_route != 0:
        count = count + 1
    if route_1 != 0:
        count = count + 1
    if route_2 != 0:
        routes = route_2.split("/")
        count = count + len(routes)
    cost = df.sal_cost / count
    return cost, count


sal_df[['per_run_sal', "count_of"]] = sal_df.apply(per_route_cost, axis=1, result_type='expand') 

sal_df_1 = sal_df[sal_df.Primary_route.ne(
    0)][["Date", "Primary_route", "per_run_sal", "count_of"]].rename(columns={'Primary_route': 'route'})

sal_df_2 = sal_df[sal_df.Satellite_Route_1.ne(
    0)][["Date", "Satellite_Route_1", "per_run_sal", "count_of"]].rename(columns={'Satellite_Route_1': 'route'})

sal_df_3 = (sal_df[sal_df.Satellite_Route_2.ne(0)][["Date", "Satellite_Route_2", "per_run_sal", "count_of"]]
            .set_index(['Date', 'per_run_sal', 'count_of'])
            .apply(lambda x: x['Satellite_Route_2']
            .split('/') if type(x['Satellite_Route_2']) == str else 0, axis=1)
            .explode()
            .reset_index()
            .rename(columns={ 0 : 'route'})
            )

pd.concat([sal_df_1,sal_df_2,sal_df_3])


,Date,route,per_run_sal,count_of
0,2021-09-01,RLC,0.000000,1.0
1,2021-09-01,RLK,619.383214,1.0
2,2021-09-01,BR1,220.935714,1.0
3,2021-09-01,SWG,328.550000,1.0
4,2021-09-01,HYG,153.338750,4.0
...,...,...,...,...
0,2021-09-01,RL6,153.338750,4.0
1,2021-09-01,RLP,153.338750,4.0
2,2021-09-02,RLG,140.435000,3.0
3,2021-09-07,RL6,153.338750,4.0


In [66]:
pd.concat([sal_df_1, sal_df_2,sal_df_3,sal_df_4]).per_run_sal.sum()

46120.6225

In [88]:
count_dr_shifts

,pri_dr,sec_dr
ir02,3.0,1.0
ir12,2.0,4.0
irdeaz,5.0,0.0
subcon1,3.0,0.0
swansgia,5.0,0.0
swbranat,5.0,1.0
swbraric,4.0,0.0
swburlei,4.0,0.0
swdinvie,5.0,0.0
swedwben,5.0,0.0


In [90]:
sal_shifts_df


,Employee_ID,Hours,Gross,Super,total,total_shifts
0,swansgia,55.10,2278.21,120.24,2398.45,5.0
1,swbranat,42.05,1270.39,103.36,1373.75,6.0
2,swbraric,30.65,1252.59,114.74,1367.33,4.0
3,swburlei,41.42,1134.14,94.77,1228.91,4.0
4,swdinvie,40.70,1403.06,123.19,1526.25,5.0
5,swedwben,49.35,1543.96,98.79,1642.75,5.0
6,swfiejih,32.65,1133.48,130.24,1263.72,3.0
7,swgooowe,0.00,1504.00,0.00,1504.00,NaN
8,swhearay,54.00,1502.09,91.70,1593.79,4.0
9,swhewg,17.85,520.32,79.04,599.36,1.0


In [94]:
sal_shifts_df

,Employee_ID,Hours,Gross,Super,total,total_shifts,per_shift_pay
0,swansgia,55.10,2278.21,120.24,2398.45,5.0,479.690000
1,swbranat,42.05,1270.39,103.36,1373.75,6.0,228.958333
2,swbraric,30.65,1252.59,114.74,1367.33,4.0,341.832500
3,swburlei,41.42,1134.14,94.77,1228.91,4.0,307.227500
4,swdinvie,40.70,1403.06,123.19,1526.25,5.0,305.250000
5,swedwben,49.35,1543.96,98.79,1642.75,5.0,328.550000
6,swfiejih,32.65,1133.48,130.24,1263.72,3.0,421.240000
7,swgooowe,0.00,1504.00,0.00,1504.00,NaN,NaN
8,swhearay,54.00,1502.09,91.70,1593.79,4.0,398.447500
9,swhewg,17.85,520.32,79.04,599.36,1.0,599.360000


In [106]:
sal_df.sal_cost.sum()

45673.51

In [101]:
roster_df.pri_sal.sum()

40805.0775

In [102]:
roster_df.sec_sal.sum()

4868.4325

In [103]:
# pd.concat([sal_df_1,sal_df_2,sal_df_3,sal_df_4])

In [110]:
sal_df

,Date,sal_cost,Primary_route,Satellite_Route_1,Satellite_Route_2
0,2021-09-01,0.000000,RLC,0,0
1,2021-09-01,619.383214,RLK,0,0
2,2021-09-01,220.935714,BR1,0,0
3,2021-09-01,328.550000,SWG,0,0
4,2021-09-01,613.355000,HYG,RL5,RL6/RLP
...,...,...,...,...,...
107,2021-09-07,288.940000,NIWPR1,0,0
108,2021-09-07,431.345000,BR3,0,0
109,2021-09-07,518.490952,RLD,0,0
110,2021-09-07,518.490952,RLJ,0,0


In [ ]:

# emp_id = df.Secondary_employeeID
# sal = sal_per_shift.loc[sal_per_shift.Employee_ID == emp_id].per_shift_pay.values


# .T.fillna(0)

# roster_df.Satellite_Route_2
